In [7]:
!wc -l pd_sampleids

2104 pd_sampleids


In [ ]:
!gsutil -u $GOOGLE_PROJECT -m cp gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/aux/ancestry/ancestry_preds.tsv .

In [8]:
!for i in `cat pd_sampleids` ; do grep -w "$i" ancestry_preds.tsv >> pd_ancestry ; done

In [ ]:
!for i in `cat lrrk2_final_controls` ; do grep -w "$i" ancestry_preds.tsv >> lrrk_control_ancestry ; done

In [7]:
import os
bucket = os.getenv("WORKSPACE_BUCKET")
bucket
import hail as hl
hl.default_reference(new_default_reference = "GRCh38")
vds_srwgs_path = os.getenv("WGS_VDS_PATH")
vds_srwgs_path
vds_srwgs_path = "gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/vds/hail.vds"
vds = hl.vds.read_vds(vds_srwgs_path)
test_intervals = ['chr12:40224997-40369285']
vds = hl.vds.filter_intervals(
    vds,
    [hl.parse_locus_interval(x,)
     for x in test_intervals])

Loading BokehJS ...

Initializing Hail with default parameters...
/opt/conda/lib/python3.10/site-packages/hailtop/aiocloud/aiogoogle/user_config.py:43: UserWarning:

Reading spark-defaults.conf to determine GCS requester pays configuration. This is deprecated. Please use `hailctl config set gcs_requester_pays/project` and `hailctl config set gcs_requester_pays/buckets`.

Running on Apache Spark version 3.5.3
SparkUI available at http://all-of-us-4100-m.us-central1-c.c.terra-vpc-sc-177a91dc.internal:43479
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.134-952ae203dbbe
LOGGING: writing to /home/jupyter/workspaces/geneticsofneurologicalandmovementdiseases/hail-20250606-0126-0.2.134-952ae203dbbe.log


In [8]:
vds.variant_data.count()

(60514, 414830)

In [9]:
mt = vds.variant_data.annotate_entries(AD = hl.vds.local_to_global(vds.variant_data.LAD, 
                                                                   vds.variant_data.LA, 
                                                                   n_alleles=hl.len(vds.variant_data.alleles), 
                                                                   fill_value=0, number='R'))

In [10]:
mt = mt.annotate_entries(GT = hl.vds.lgt_to_gt(mt.LGT, mt.LA))
mt = mt.transmute_entries(FT = hl.if_else(mt.FT, "PASS", "FAIL"))
mt = hl.vds.to_dense_mt(hl.vds.VariantDataset(vds.reference_data, mt))
mt = mt.annotate_rows(info = hl.agg.call_stats(mt.GT, mt.alleles))
fields_to_drop_list = ['as_vets','as_vqsr','LAD', 'LGT', 'LA',
            'tranche_data', 'truth_sensitivity_snp_threshold', 
             'truth_sensitivity_indel_threshold','snp_vqslod_threshold','indel_vqslod_threshold']

In [11]:
mt = mt.drop(*(f for f in fields_to_drop_list if f in mt.entry or f in mt.row or f in mt.col or f in mt.globals))

In [12]:
out_path = f'{bucket}/data/test_hail.mt'
mt.write(out_path, overwrite = True)

2025-05-04 00:53:20.360 Hail: INFO: wrote matrix table with 60514 rows and 414830 columns in 9 partitions to gs://fc-secure-37344552-0ee7-4c55-a79b-eecb5463419a/data/test_hail.mt


In [13]:
mt_vcf = mt.transmute_rows(info = mt.info.annotate(AF=mt.info.AF[1:], 
                                                              AC=mt.info.AC[1:]))

In [ ]:
vcf_header = "gs://fc-secure-ff68a895-e88d-426e-9ae4-b3802c51b53b/data/vcf_header.txt"
metadata = hl.get_vcf_metadata(vcf_header)
out_vcf = f'{bucket}/data/lrrk2_vcf.vcf.bgz'
hl.export_vcf(mt_vcf, out_vcf, tabix = False, metadata=metadata)

In [2]:
import os
bucket = os.getenv("WORKSPACE_BUCKET")
bucket
!gsutil -u $GOOGLE_PROJECT -m cp '{bucket}/data/lrrk2_vcf.vcf.bgz' .

Copying gs://fc-secure-37344552-0ee7-4c55-a79b-eecb5463419a/data/lrrk2_vcf.vcf.bgz...
/ [1/1 files][  1.5 GiB/  1.5 GiB] 100% Done  57.0 MiB/s ETA 00:00:00           
Operation completed over 1 objects/1.5 GiB.                                      


In [12]:
!bcftools annotate --header-lines <(echo '##FILTER=<ID=EXCESS_ALLELES,Description="More than 3 alleles">') \
-Oz -o lrrk2_fixed.vcf.bgz lrrk2_vcf.vcf.bgz

In [13]:
!bcftools view -S pd_sampleids -Oz -o pdsamples_lrrk2.vcf.bgz lrrk2_fixed.vcf.bgz --force-samples
!bcftools view -S lrrk2_controlids -Oz -o controls_lrrk2.vcf.bgz lrrk2_fixed.vcf.bgz --force-samples

Warn: subset called for sample that does not exist in header: "person_id"... skipping


In [14]:
!bcftools norm -m -both -Oz -o pdfiltered_split_lrrk2.vcf.bgz -f Homo_sapiens_assembly38.fasta pdsamples_lrrk2.vcf.bgz
!bcftools norm -m -both -Oz -o controls_lrrk2_split.vcf.bgz -f Homo_sapiens_assembly38.fasta controls_lrrk2.vcf.bgz

Lines   total/split/realigned/skipped:	60514/11631/6115/0


In [15]:
!tabix -p vcf pdfiltered_split_lrrk2.vcf.bgz
!tabix -p vcf controls_lrrk2_split.vcf.bgz

In [16]:
!zgrep -v "contig" pdfiltered_split_lrrk2.vcf.bgz | \
awk -F'\t' '$0 ~ /^chr12/ && $0 !~ /AC=0;/ && $0 !~ /AF=0\.[56789]/' > lrrk2_variants
!zgrep -v "contig" controls_lrrk2_split.vcf.bgz | \
awk -F'\t' '$0 ~ /^chr12/ && $0 !~ /AC=0;/ && $0 !~ /AF=0\.[56789]/' >> lrrk2_variants

In [ ]:
#Adding the header to create a vcf file
!zcat pdfiltered_split_lrrk2.vcf.bgz | head -3390 > lrrk2_header
!cat lrrk2_header lrrk2_variants > lrrk2_variants.vcf

In [ ]:
#Annotation was performed in NIH's Biowulf HPC using the following parameters:
module load annovar
table_annovar.pl {WORK_DIR}/lrrk2_variants.vcf $ANNOVAR_DATA/hg38 \
--buildver hg38 -out {WORK_DIR}/lrrk2_variants_anno -remove \
-protocol gene,clinvar_20240611,dbnsfp47a,ljb26_all,avsnp151,gnomad41_genome,gnomad41_exome --operation g,f,f,f,f,f,f  \
-nastring . -vcfinput 
